In [1]:
spark

In [25]:
#import statements
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import *
from datetime import datetime

In [26]:
spark = SparkSession.builder.appName('Final_project').getOrCreate()
sc = spark.sparkContext

In [27]:
#Datapath
path_comments = "/user/rroongseang/bigdata/comments/"
path_users = "/user/rroongseang/bigdata/users/"

In [28]:
#Read data
df = spark.read.json(path_comments+"politics_json*")
users = spark.read.csv(path_users+"RA.2019-09.csv", inferSchema=True, header=True)

In [29]:
#Convert column types to integers and timestamps
from pyspark.sql.types import IntegerType
df = df.withColumn("ups", df["ups"].cast(IntegerType())) 
df = df.withColumn("downs", df["downs"].cast(IntegerType()))
df = df.withColumn("gilded", df["gilded"].cast(IntegerType()))
df = df.withColumn("score", df["score"].cast(IntegerType()))
df = df.withColumn("retrieved_on", df["retrieved_on"].cast(IntegerType()))
df = df.withColumn("controversiality", df["controversiality"].cast(IntegerType()))
df = df.withColumn('created_utc',df["created_utc"].cast(IntegerType()))
df = df.withColumn('retrieved_on',df["retrieved_on"].cast(IntegerType()))


df = df.withColumn('created_utc',to_timestamp(df["created_utc"]))
df = df.withColumn('retrieved_on',to_timestamp(df["retrieved_on"]))

In [30]:
df.printSchema()

root
 |-- archived: boolean (nullable = true)
 |-- author: string (nullable = true)
 |-- author_flair_css_class: string (nullable = true)
 |-- author_flair_text: string (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: integer (nullable = true)
 |-- created_utc: timestamp (nullable = true)
 |-- distinguished: string (nullable = true)
 |-- downs: integer (nullable = true)
 |-- gilded: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- link_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- retrieved_on: timestamp (nullable = true)
 |-- score: integer (nullable = true)
 |-- score_hidden: boolean (nullable = true)
 |-- subreddit: string (nullable = true)
 |-- subreddit_id: string (nullable = true)
 |-- ups: integer (nullable = true)



In [31]:
#Rename a couple user columns to not duplicate with comments dataframe
users = users.select("id", col("name").alias("username"),
                     col("created_utc").alias("acct_creation"),"updated_on","comment_karma","link_karma")

In [32]:
#Convert account creation and updated on to timestampps
users = users.withColumn('acct_creation',to_timestamp(users["acct_creation"]))
users = users.withColumn('updated_on',to_timestamp(users["updated_on"]))

In [33]:
users.printSchema()

root
 |-- id: long (nullable = true)
 |-- username: string (nullable = true)
 |-- acct_creation: timestamp (nullable = true)
 |-- updated_on: timestamp (nullable = true)
 |-- comment_karma: integer (nullable = true)
 |-- link_karma: integer (nullable = true)



In [52]:
#Checking to see if one of the Russian campaigns is in the users list
users.filter(users.username == 'gregoratior').show()

#I tested out 3 different usernames and none of them showed up. I will have to run the scraper which pulls specific user info

+---+--------+-------------+----------+-------------+----------+
| id|username|acct_creation|updated_on|comment_karma|link_karma|
+---+--------+-------------+----------+-------------+----------+
+---+--------+-------------+----------+-------------+----------+



In [53]:
#Checking to see if one of the Russian comments show up in the comments.
df.filter(df.author == 'gregoratior').show()

#There are 50 other comments files so hopefully their comments show up in one of the other ones.

+--------+------+----------------------+-----------------+----+----------------+-----------+-------------+-----+------+---+-------+----+---------+------------+-----+------------+---------+------------+---+
|archived|author|author_flair_css_class|author_flair_text|body|controversiality|created_utc|distinguished|downs|gilded| id|link_id|name|parent_id|retrieved_on|score|score_hidden|subreddit|subreddit_id|ups|
+--------+------+----------------------+-----------------+----+----------------+-----------+-------------+-----+------+---+-------+----+---------+------------+-----+------------+---------+------------+---+
+--------+------+----------------------+-----------------+----+----------------+-----------+-------------+-----+------+---+-------+----+---------+------------+-----+------------+---------+------------+---+



In [54]:
#Combining data frames
df_combined = df.join(users, df.author == users.username, 'left')

In [46]:
#show sample combined dataframe
df_combined.select(df.author,df.body,df.ups,df.downs,df.created_utc,users.acct_creation,users.username).sample(False,.00001,3).show(5)

+-------------+--------------------+----+-----+-------------------+-------------------+-------------+
|       author|                body| ups|downs|        created_utc|      acct_creation|     username|
+-------------+--------------------+----+-----+-------------------+-------------------+-------------+
|  sparebutter|we as a workforce...|null| null|2019-01-15 08:04:25|               null|         null|
|Dux_Ignobilis|I say this as som...|null| null|2016-11-21 09:18:18|2015-01-18 22:17:40|Dux_Ignobilis|
|  drpinkcream|Those who can’t f...|null| null|2017-07-27 16:27:06|2011-05-01 07:15:05|  drpinkcream|
|    [deleted]|           [removed]|null| null|2018-04-27 08:04:00|               null|         null|
|    [deleted]|           [removed]|null| null|2018-08-15 09:23:28|               null|         null|
+-------------+--------------------+----+-----+-------------------+-------------------+-------------+
only showing top 5 rows



In [47]:
#Number of usernames matched 
df_combined.filter(users.username.isNotNull()).count()

5158836

In [56]:
#number of comments in original df
df.count()

6420881

In [48]:
#number of comments in combined df
df_combined.count()

6420881